In [1]:
import os
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [2]:
def remove_background(image):
    """
    Removes the background of an image using the GrabCut algorithm.
    Assumes the pig is the main object in the image.
    """
    mask = np.zeros(image.shape[:2], np.uint8)
    
    # Define background and foreground models
    bgd_model = np.zeros((1, 65), np.float64)
    fgd_model = np.zeros((1, 65), np.float64)
    
    # Define a rectangle around the object (assumed to be centered)
    height, width = image.shape[:2]
    
    # Ensure the rectangle is within valid bounds
    x1, y1 = max(1, width // 20), max(1, height // 20)
    x2, y2 = min(width - 1, width - x1), min(height - 1, height - y1)

    if x2 <= x1 or y2 <= y1:
        print(f"Skipping image due to invalid dimensions: {width}x{height}")
        return None, None  # Return None to handle errors gracefully

    rect = (x1, y1, x2 - x1, y2 - y1)  # Safe rectangle

    # Apply GrabCut algorithm
    cv2.grabCut(image, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)

    # Convert mask to binary: 1 for foreground, 0 for background
    mask_binary = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # Create an RGBA image with a transparent background
    result = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)  # Convert to 4-channel (RGBA)
    result[:, :, 3] = mask_binary * 255  # Apply transparency mask

    return result, mask_binary * 255  # Return mask for depth processing

In [3]:
def apply_mask_to_depth(depth_image, mask):
    """
    Applies the same background removal mask to the depth image.
    """
    result = cv2.bitwise_and(depth_image, depth_image, mask=mask)

    result_bgra = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)

    # Create a mask where black pixels ([0,0,0]) are detected
    black_pixels = np.all(result[:, :, :3] == [0, 0, 0], axis=-1)

    # Set alpha channel to 0 (transparent) where black pixels are found
    result_bgra[black_pixels, 3] = 0

    return result_bgra

In [4]:
def process_subfolder(subdir, output_subdir):
    output_subdir.mkdir(parents=True, exist_ok=True)

    image_files = sorted(subdir.glob("frame_*_rgb_crop.png"))  # Assuming filenames have '_rgb.png'
    for rgb_file in tqdm(image_files, desc=f"Processing {subdir.name}"):
        depth_file = rgb_file.with_name(rgb_file.stem.replace("_rgb", "_depth") + rgb_file.suffix)

        if not depth_file.exists():
            print(f"Warning: No corresponding depth image for {rgb_file.name}")
            continue

        # Read images
        rgb_image = cv2.imread(str(rgb_file))
        depth_image = cv2.imread(str(depth_file), cv2.IMREAD_UNCHANGED)

        # Remove background
        rgb_no_bg, mask_binary = remove_background(rgb_image)
        depth_no_bg = apply_mask_to_depth(depth_image, mask_binary)

        # Save output images
        cv2.imwrite(str(output_subdir / rgb_file.name), rgb_no_bg)
        cv2.imwrite(str(output_subdir / depth_file.name), depth_no_bg)

In [5]:
def process_images(input_folder, output_folder):
    input_path = Path(input_folder)
    output_path = Path(output_folder)
    
    if not input_path.exists():
        print("Input folder does not exist.")
        return
    
    for subdir in tqdm(list(input_path.glob('*')), desc="Processing folders"):
        if subdir.is_dir():
            process_subfolder(subdir, output_path / subdir.relative_to(input_path))

In [6]:
if __name__ == "__main__":
    input_folder = "C:\\Users\\USER\\Downloads\\Thesis\\Image Processing\\Jan 14 2025 - Batch 1\\outputs_test"  # Change this to your actual path
    output_folder = ".\\Jan 14 2025 - Batch 1\\background_remove_output_20250309"
    process_images(input_folder, output_folder)

Processing folders: 100%|██████████| 7/7 [8:24:25<00:00, 4323.58s/it]  
